In [ ]:
import numpy as np
import imageio
import cv2
import glob

# Parameters
height, width, channels = 256, 256, 3

# Prepare ISIC 2018 data set
Dataset_add = '/kaggle/input/isic2018-challenge-task1-data-segmentation/'
Tr_add = 'ISIC2018_Task1-2_Training_Input'

Tr_list = glob.glob(Dataset_add + Tr_add + '/*.jpg')
Data_train_2018 = np.zeros([len(Tr_list), height, width, channels])
Label_train_2018 = np.zeros([len(Tr_list), height, width])

print('Reading ISIC 2018')
for idx, img_path in enumerate(Tr_list):
    print(idx + 1)
    img = imageio.imread(img_path)
    img = cv2.resize(img, (width, height))
    Data_train_2018[idx] = img

    # Extract corresponding mask path
    img_name = img_path.split('/')[-1].replace('.jpg', '_segmentation.png')
    mask_path = Dataset_add + 'ISIC2018_Task1_Training_GroundTruth/' + img_name
    
    img2 = imageio.imread(mask_path)
    img2 = cv2.resize(img2, (width, height))
    Label_train_2018[idx] = img2 
         
print('Reading ISIC 2018 finished')

# Split the data
Train_img, Validation_img, Test_img = np.split(Data_train_2018, [1815, 1815 + 259])
Train_mask, Validation_mask, Test_mask = np.split(Label_train_2018, [1815, 1815 + 259])

# Save the data
np.save('data_train', Train_img)
np.save('data_test', Test_img)
np.save('data_val', Validation_img)

np.save('mask_train', Train_mask)
np.save('mask_test', Test_mask)
np.save('mask_val', Validation_mask)


In [ ]:
import tensorflow as tf
device_name=tf.test.gpu_device_name()
if "GPU" not in device_name:
    print("GPU not found")
print("found gpu at: {}".format(device_name))
print("GPU","available (YES!!!!)") if tf.config.list_physical_devices("GPU") else "not available:("

convert masks in to binary format save them as in a new nympy array 

In [ ]:
import numpy as np

# Load masks
train_masks = np.load('mask_train.npy')
val_masks = np.load('mask_val.npy')
test_masks = np.load('mask_test.npy')

# Ensure masks are binary (thresholding)
train_masks = (train_masks > 0.5).astype(np.uint8)
val_masks = (val_masks > 0.5).astype(np.uint8)
test_masks = (test_masks > 0.5).astype(np.uint8)

# Save the processed binary masks
np.save('mask_train_binary.npy', train_masks)
np.save('mask_val_binary.npy', val_masks)
np.save('mask_test_binary.npy', test_masks)

print("Masks have been successfully binarized and saved.")


normalize images from 0-255 to 0-1 and covert float32 and save them in new numpy array(normalized)

In [ ]:
import numpy as np

# Load the original images and masks (in 0-255 format)
train_images = np.load('data_train.npy')
val_images = np.load('data_val.npy')
test_images = np.load('data_test.npy')

# Normalize the images by dividing by 255 (convert to float32)
train_images_normalized = train_images.astype(np.float32) / 255.0
val_images_normalized = val_images.astype(np.float32) / 255.0
test_images_normalized = test_images.astype(np.float32) / 255.0

# Save the normalized numpy arrays to new files
np.save('data_train_normalized.npy', train_images_normalized)
np.save('data_val_normalized.npy', val_images_normalized)
np.save('data_test_normalized.npy', test_images_normalized)

print("Normalization and saving completed.")


In [ ]:
# ===== normalize over the dataset 
def dataset_normalized(imgs):
    imgs_normalized = np.empty(imgs.shape)
    imgs_std = np.std(imgs)
    imgs_mean = np.mean(imgs)
    imgs_normalized = (imgs-imgs_mean)/imgs_std
    for i in range(imgs.shape[0]):
        imgs_normalized[i] = ((imgs_normalized[i] - np.min(imgs_normalized[i])) / (np.max(imgs_normalized[i])-np.min(imgs_normalized[i])))
    return imgs_normalized
       

normalizing way two same as that of the paper

In [ ]:
import numpy as np
# Load the original images and masks (in 0-255 format)
train_images = np.load('/kaggle/input/isic-2018-preprocessed/data_train.npy')
val_images = np.load('/kaggle/input/isic-2018-preprocessed/data_val.npy')
test_images = np.load('/kaggle/input/isic-2018-preprocessed/data_test.npy')

# Normalize the images by dividing by 255 (convert to float32)
train_images_normalized1 = dataset_normalized(train_images.astype(np.float32)) 
val_images_normalized1 = dataset_normalized(val_images.astype(np.float32)) 
test_images_normalized1 = dataset_normalized(test_images.astype(np.float32))

# Save the normalized numpy arrays to new files
np.save('data_train_normalized1.npy', train_images_normalized1)
np.save('data_val_normalized1.npy', val_images_normalized1)
np.save('data_test_normalized1.npy', test_images_normalized1)

print("Normalization and saving completed.")

veryfing masks are binary and printing sample data

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# Load the binary masks
train_images = np.load('data_train_normalized1.npy')
train_masks = np.load('/kaggle/input/isic2018-preprocessed/mask_train_binary.npy')
val_images = np.load('data_val_normalized1.npy')
val_masks = np.load('/kaggle/input/isic2018-preprocessed/mask_val_binary.npy')
test_images = np.load('data_test_normalized1.npy')
test_masks = np.load('/kaggle/input/isic2018-preprocessed/mask_test_binary.npy')

# Check shapes
print("Train Images Shape: ", train_images.shape)
print("Train Masks Shape: ", train_masks.shape)
print("Validation Images Shape: ", val_images.shape)
print("Validation Masks Shape: ", val_masks.shape)
print("Test Images Shape: ", test_images.shape)
print("Test Masks Shape: ", test_masks.shape)

# Visualize a few samples to check if the images and masks are correct
def display_sample(images, masks, index):
    """Displays an image and its corresponding mask."""
    image = images[index]
    mask = masks[index]

    fig, ax = plt.subplots(1, 2, figsize=(10, 5))
    ax[0].imshow(image)
    ax[0].set_title('Image')
    ax[1].imshow(mask)
    ax[1].set_title('Mask')
    plt.show()

# Display a random training sample
display_sample(train_images, train_masks, index=0)

# Check that masks are binary
assert np.array_equal(np.unique(train_masks), [0, 1]), "Train masks are not binary!"
assert np.array_equal(np.unique(val_masks), [0, 1]), "Validation masks are not binary!"
assert np.array_equal(np.unique(test_masks), [0, 1]), "Test masks are not binary!"

print("Dataset verification completed successfully.")


In [ ]:
# Check min and max values of the images
print("Min pixel value in training images:", np.min(train_images))
print("Max pixel value in training images:", np.max(train_images))

# Check min and max values of a specific image
sample_image = train_images[0]
print("Min pixel value in sample image:", np.min(sample_image))
print("Max pixel value in sample image:", np.max(sample_image))


defining model unet 

In [ ]:
import numpy as np
from keras.models import Model
from keras.layers import Input, concatenate, Conv2D, MaxPooling2D, UpSampling2D, Dropout
from keras.optimizers import Adam
from keras.layers import BatchNormalization
#import os
#os.environ["TF_FORCE_GPU_ALLOW_GROWTH"] = "true"

def unet(input_size=(256, 256, 3)):  # Changed to 3 channels for RGB
    inputs = Input(input_size)

    # Downsampling path
    conv1 = Conv2D(64, 3, activation='relu', padding='same', kernel_initializer='he_normal')(inputs)
    conv1 = Conv2D(64, 3, activation='relu', padding='same', kernel_initializer='he_normal')(conv1)
    conv1 = BatchNormalization()(conv1)
    pool1 = MaxPooling2D(pool_size=(2, 2))(conv1)
    
    conv2 = Conv2D(128, 3, activation='relu', padding='same', kernel_initializer='he_normal')(pool1)
    conv2 = Conv2D(128, 3, activation='relu', padding='same', kernel_initializer='he_normal')(conv2)
    conv2 = BatchNormalization()(conv2)
    pool2 = MaxPooling2D(pool_size=(2, 2))(conv2)
    
    conv3 = Conv2D(256, 3, activation='relu', padding='same', kernel_initializer='he_normal')(pool2)
    conv3 = Conv2D(256, 3, activation='relu', padding='same', kernel_initializer='he_normal')(conv3)
    conv3 = BatchNormalization()(conv3)
    drop3 = Dropout(0.5)(conv3)
    pool3 = MaxPooling2D(pool_size=(2, 2))(drop3)
    
    conv4 = Conv2D(512, 3, activation='relu', padding='same', kernel_initializer='he_normal')(pool3)
    conv4 = Conv2D(512, 3, activation='relu', padding='same', kernel_initializer='he_normal')(conv4)
    conv4 = BatchNormalization()(conv4)
    drop4 = Dropout(0.5)(conv4)

    # Upsampling path
    up6 = Conv2D(512, 2, activation='relu', padding='same', kernel_initializer='he_normal')(UpSampling2D(size=(2, 2))(drop4))
    merge6 = concatenate([drop3, up6], axis=3)
    conv6 = Conv2D(256, 3, activation='relu', padding='same', kernel_initializer='he_normal')(merge6)
    conv6 = Conv2D(256, 3, activation='relu', padding='same', kernel_initializer='he_normal')(conv6)
    conv6 = BatchNormalization()(conv6)

    up7 = Conv2D(256, 2, activation='relu', padding='same', kernel_initializer='he_normal')(UpSampling2D(size=(2, 2))(conv6))
    merge7 = concatenate([conv2, up7], axis=3)
    conv7 = Conv2D(128, 3, activation='relu', padding='same', kernel_initializer='he_normal')(merge7)
    conv7 = Conv2D(128, 3, activation='relu', padding='same', kernel_initializer='he_normal')(conv7)
    conv7 = BatchNormalization()(conv7)

    up8 = Conv2D(64, 2, activation='relu', padding='same', kernel_initializer='he_normal')(UpSampling2D(size=(2, 2))(conv7))
    merge8 = concatenate([conv1, up8], axis=3)
    conv8 = Conv2D(64, 3, activation='relu', padding='same', kernel_initializer='he_normal')(merge8)
    conv8 = Conv2D(64, 3, activation='relu', padding='same', kernel_initializer='he_normal')(conv8)
    conv8 = BatchNormalization()(conv8)
    
    # Output layer for binary segmentation
    conv9 = Conv2D(1, 1, activation='sigmoid')(conv8)

    # Define the model
    model = Model(inputs=inputs, outputs=conv9)
    
    # Compile the model
    model.compile(optimizer=Adam(learning_rate=1e-4), loss='binary_crossentropy', metrics=['accuracy'])

    return model


In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# Load the datasets
train_images = np.load('data_train_normalized.npy')
train_masks = np.load('mask_train_binary.npy')
val_images = np.load('data_val_normalized.npy')
val_masks = np.load('mask_val_binary.npy')
test_images = np.load('data_test_normalized.npy')
test_masks = np.load('mask_test_binary.npy')

# Check shapes
print("Train Images Shape: ", train_images.shape)
print("Train Masks Shape: ", train_masks.shape)
print("Validation Images Shape: ", val_images.shape)
print("Validation Masks Shape: ", val_masks.shape)
print("Test Images Shape: ", test_images.shape)
print("Test Masks Shape: ", test_masks.shape)

# Ensure the image shape is (256, 256, 3) and the mask shape is (256, 256)
assert train_images.shape[1:] == (256, 256, 3), "Training images shape mismatch!"
assert train_masks.shape[1:] == (256, 256), "Training masks shape mismatch!"

# Visualize a few samples to check if the images and masks are correct
def display_sample(images, masks, index):
    """Displays an image and its corresponding mask."""
    image = images[index]
    mask = masks[index]


    fig, ax = plt.subplots(1, 2, figsize=(10, 5))
    ax[0].imshow(image)
    ax[0].set_title('Image')
    ax[1].imshow(mask)
    ax[1].set_title('Mask')
    plt.show()

# Display a random training sample
display_sample(train_images, train_masks, index=0)

# Check that masks are binary
assert np.array_equal(np.unique(train_masks), [0, 1]), "Train masks are not binary!"
assert np.array_equal(np.unique(val_masks), [0, 1]), "Validation masks are not binary!"
assert np.array_equal(np.unique(test_masks), [0, 1]), "Test masks are not binary!"

print("Dataset verification completed successfully.")


above is all preprocessing the dataset which already loaded in the datasets(input tab)

In [ ]:
# Ensure that masks have shape (256, 256, 1)
train_masks = np.expand_dims(train_masks, axis=-1)
val_masks = np.expand_dims(val_masks, axis=-1)
test_masks = np.expand_dims(test_masks, axis=-1)

In [ ]:

# Check shapes
print("Train Images Shape: ", train_images.shape)
print("Train Masks Shape: ", train_masks.shape)
print("Validation Images Shape: ", val_images.shape)
print("Validation Masks Shape: ", val_masks.shape)
print("Test Images Shape: ", test_images.shape)
print("Test Masks Shape: ", test_masks.shape)

train

In [ ]:
from keras.callbacks import ModelCheckpoint, EarlyStopping,ReduceLROnPlateau
# Instantiate the model
model = unet(input_size=(256, 256, 3))

# ModelCheckpoint to save the entire model (architecture + weights)
checkpoint = ModelCheckpoint('unet_model.keras', save_best_only=True, monitor='val_loss', mode='min', verbose=1)

# ReduceLROnPlateau to reduce learning rate when validation loss plateaus
reduce_lr_loss = ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=7, verbose=1, min_delta=1e-4, mode='min')

# EarlyStopping to stop training if validation loss doesn't improve for 10 consecutive epochs
early_stopping = EarlyStopping(monitor='val_loss', patience=10, verbose=1, mode='min')

# Train the model with the modified setup
history = model.fit(
    train_images, train_masks,
    validation_data=(val_images, val_masks),
    batch_size=16,
    epochs=25,
    shuffle=True,
    callbacks=[checkpoint, reduce_lr_loss, early_stopping],
    verbose=1
)


# Save the final trained model
model.save('unet_final_model.keras')

print("Training completed successfully.")

In [ ]:
from keras.callbacks import ModelCheckpoint, EarlyStopping,ReduceLROnPlateau
# Instantiate the model
model = unet(input_size=(256, 256, 3))

# ModelCheckpoint to save the entire model (architecture + weights)
checkpoint = ModelCheckpoint('unet_model100ep.keras', save_best_only=True, monitor='val_loss', mode='min', verbose=1)

# ReduceLROnPlateau to reduce learning rate when validation loss plateaus
reduce_lr_loss = ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=7, verbose=1, min_delta=1e-4, mode='min')

# EarlyStopping to stop training if validation loss doesn't improve for 10 consecutive epochs
early_stopping = EarlyStopping(monitor='val_loss', patience=10, verbose=1, mode='min')

# Train the model with the modified setup
history = model.fit(
    train_images, train_masks,
    validation_data=(val_images, val_masks),
    batch_size=8,
    epochs=100,
    shuffle=True,
    callbacks=[checkpoint, reduce_lr_loss],
    verbose=1
)


# Save the final trained model
model.save('unet_final_model_100ep.keras')

print("Training completed successfully.")

In [ ]:
import matplotlib.pyplot as plt

# Function to plot training and validation metrics
def plot_training_history(history):
    # Plot training & validation accuracy values
    plt.figure(figsize=(12, 4))
    
    # Subplot for accuracy
    plt.subplot(1, 2, 1)
    plt.plot(history.history['accuracy'], label='Training Accuracy')
    plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
    plt.title('Model Accuracy')
    plt.xlabel('Epoch')
    plt.ylabel('Accuracy')
    plt.legend(loc='upper left')
    
    # Subplot for loss
    plt.subplot(1, 2, 2)
    plt.plot(history.history['loss'], label='Training Loss')
    plt.plot(history.history['val_loss'], label='Validation Loss')
    plt.title('Model Loss')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.legend(loc='upper left')
    
    # Display the plots
    plt.tight_layout()
    plt.show()

# Call the function to plot the training history
plot_training_history(history)


evaluation--matrixs

In [ ]:
import numpy as np
from keras.models import load_model
from sklearn.metrics import accuracy_score, f1_score, jaccard_score, confusion_matrix



# Load the trained model
model = load_model('unet_final_model_100ep.keras')

# Make predictions on the test set
predictions = model.predict(test_images)

# Binarize predictions by applying a threshold of 0.5
predictions = (predictions > 0.5).astype(np.uint8)

# Flatten the masks and predictions for evaluation
test_masks_flat = test_masks.flatten()
predictions_flat = predictions.flatten()

# Calculate evaluation metrics
accuracy = accuracy_score(test_masks_flat, predictions_flat)
f1 = f1_score(test_masks_flat, predictions_flat)
iou = jaccard_score(test_masks_flat, predictions_flat)
conf_matrix = confusion_matrix(test_masks_flat, predictions_flat)

# Print the results
print(f"Accuracy: {accuracy:.4f}")
print(f"F1 Score: {f1:.4f}")
print(f"Jaccard Index (IoU): {iou:.4f}")
print("Confusion Matrix:")
print(conf_matrix)


result --generated mask,true mask,image

In [ ]:
import matplotlib.pyplot as plt

# Function to display a sample test image, predicted mask, and true mask
def display_test_sample(images, true_masks, predicted_masks, index):
    """Displays test image, true mask, and predicted mask side by side."""
    image = images[index]
    true_mask = true_masks[index]
    predicted_mask = predicted_masks[index]

    fig, ax = plt.subplots(1, 3, figsize=(15, 5))
    ax[0].imshow(image)
    ax[0].set_title('Test Image')

    ax[1].imshow(true_mask, cmap='gray')
    ax[1].set_title('True Mask')

    ax[2].imshow(predicted_mask, cmap='gray')
    ax[2].set_title('Predicted Mask')

    plt.show()

# Display a few test samples along with their true and predicted masks
for i in range(15):  # Display the first 3 samples
    display_test_sample(test_images, test_masks, predictions, i)


exact copy of the implemented in paper